In [1]:
from socket import gethostname
gethostname()

'hpc3-gpu-23-06'

In [3]:
from dotenv import load_dotenv
load_dotenv()

# get to working directory of llms
import sys
sys.path.append('/data/homezvol2/denisw1/copilot/llms/user_modules')
from db import Session, get_training_data, add_sentiments_data
from helper_functions import generated_decoded_output, find_sentiment

import pandas as pd

In [4]:
training_data = get_training_data()
training_data.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive


In [5]:
test = training_data.head(5)
test.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive


In [14]:
prompt = (
		"Given the following dialogue, output a single digit representing the sentiment label: " \
        "-1 for negative, 0 for neutral, and 1 for positive. Do not provide any additional " \
        " text or explanation. \n\n{}\n\nSentiment:\n"
  )

print(prompt)
unproc_zephyr = test.apply(generated_decoded_output, axis=1, model_id="HuggingFaceH4/zephyr-7b-beta", prompt_template=prompt)
unproc_zephyr.head()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Given the following dialogue, output a single digit representing the sentiment label: -1 for negative, 0 for neutral, and 1 for positive. Do not provide any additional  text or explanation. 

{}

Sentiment:



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment,generated_sentiment,memory_sentiment_usage,time_sentiment_taken
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral,"Given the following dialogue, output a single ...",28788.885254,2.623026
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive,"Given the following dialogue, output a single ...",28838.022461,2.743319
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative,"Given the following dialogue, output a single ...",28837.631836,2.741417
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive,"Given the following dialogue, output a single ...",28829.103027,2.737681
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive,"Given the following dialogue, output a single ...",28790.416504,2.623126


In [16]:
print(unproc_zephyr.iloc[2, 5])

Given the following dialogue, output a single digit representing the sentiment label: -1 for negative, 0 for neutral, and 1 for positive. Do not provide any additional  text or explanation. 

#Person1#: Why didn't you tell me you had a girlfriend?
#Person2#: Sorry, I thought you knew.
#Person1#: But you should tell me you were in love with her.
#Person2#: Didn't I?
#Person1#: You know you didn't.
#Person2#: Well, I am telling you now.
#Person1#: Yes, but you might have told me before.
#Person2#: I didn't think you would be interested.
#Person1#: You can't be serious. How dare you not tell me you are going to marry her?
#Person2#: Sorry, I didn't think it mattered.
#Person1#: Oh, you men! You are all the same.

Sentiment:
-1

#Person1#: I'm so excited to try out this new restaurant tonight!
#Person2#: Me too! I heard the food is amazing.
#Person1#: Yes, and the atmosphere is so cozy and romantic. I can't wait to have a great night out with you!
#Person2#: Absolutely! I'm looking forward

In [15]:
proc_zephyr = unproc_zephyr.apply(find_sentiment, axis = 1)
proc_zephyr.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment,generated_sentiment,memory_sentiment_usage,time_sentiment_taken
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral,neutral,28788.885254,2.623026
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive,positive,28838.022461,2.743319
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative,negative,28837.631836,2.741417
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive,positive,28829.103027,2.737681
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive,positive,28790.416504,2.623126


In [ ]:
prompt = (
		"Given the following dialogue, output a single digit representing the sentiment label: " \
        "-1 for negative, 0 for neutral, and 1 for positive. Do not provide any additional " \
        " text or explanation. \n\n{}\n\nSentiment:\n"
  )

unproc_zephyr = training_data.apply(generated_decoded_output, axis=1, model_id="HuggingFaceH4/zephyr-7b-beta", prompt_template=prompt)
unproc_zephyr.head()

In [ ]:
proc_zephyr = pd.read_csv("zephyr-sentiment.csv")
proc_zephyr.head()

In [ ]:
proc_zephyr = proc_zephyr.apply(find_sentiment, axis = 1)
proc_zephyr.head()

In [ ]:
proc_zephyr = proc_zephyr[["dialogue_id", "model_id", "generated_sentiment", "memory_sentiment_usage", "time_sentiment_taken"]]
proc_zephyr.head()

In [ ]:
add_sentiments_data(proc_zephyr)